In [1]:
import numpy as np
import pennylane as qml
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load Iris and make it binary: Setosa (1) vs Not Setosa (0)
X, y = datasets.load_iris(return_X_y=True)
y = (y == 0).astype(int)

# Use two features to match our 2-qubit feature map
# (petal length, petal width) are very discriminative
X = X[:, [2, 3]]

# Split
Xtr, Xte, ytr, yte = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

# Standardise features using training stats only
sc = StandardScaler().fit(Xtr)
Xtr_std, Xte_std = sc.transform(Xtr), sc.transform(Xte)

# Map to angles: [-pi, pi] keeps rotations well-conditioned
def to_angles(Z):
    Z = np.asarray(Z, dtype=float)
    # normalise each column by its max absolute value (avoid extreme angles)
    denom = np.maximum(np.max(np.abs(Z), axis=0), 1e-9)
    Zn = Z / denom
    return np.pi * Zn  # angles in [-pi, pi]

Xtr_ang = to_angles(Xtr_std)
Xte_ang = to_angles(Xte_std)


In [2]:
# 2-qubit exact simulator
dev = qml.device("default.qubit", wires=2, shots=None)

def feature_map(x):
    x = np.asarray(x, dtype=float)
    qml.RX(x[0], wires=0)
    qml.RX(x[1], wires=1)
    qml.CZ(wires=[0, 1])

@qml.qnode(dev)
def kernel_element(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.probs(wires=[0, 1])  # [p00, p01, p10, p11]

def quantum_kernel(XA, XB):
    XA = np.asarray(XA, dtype=float)
    XB = np.asarray(XB, dtype=float)
    K = np.zeros((len(XA), len(XB)))
    for i in range(len(XA)):
        for j in range(len(XB)):
            probs = kernel_element(XA[i], XB[j])
            K[i, j] = probs[0]  # probability of |00> = |<phi(xi)|phi(xj)>|^2
    return K


In [3]:
from sklearn.svm import SVC

Ktr = quantum_kernel(Xtr_ang, Xtr_ang)   # train-vs-train
clf = SVC(kernel="precomputed", C=1.0)
clf.fit(Ktr, ytr)

Kte = quantum_kernel(Xte_ang, Xtr_ang)   # test-vs-train
yhat = clf.predict(Kte)

acc = accuracy_score(yte, yhat)
print(f"Quantum-kernel SVM accuracy: {acc*100:.2f}%")


Quantum-kernel SVM accuracy: 100.00%


In [4]:
from sklearn.svm import SVC

rbf = SVC(kernel="rbf", C=1.0, gamma="scale").fit(Xtr_std, ytr)
acc_rbf = accuracy_score(yte, rbf.predict(Xte_std))
print(f"Classical RBF-SVM accuracy: {acc_rbf*100:.2f}%")


Classical RBF-SVM accuracy: 100.00%
